In [ ]:
# Import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import warnings

warnings.filterwarnings('ignore')

# Load the updated dataset
file_path = 'updated_medical_population_data.csv'
medical_data = pd.read_csv(file_path)

# Function to filter doctors based on top 20 flag
def filter_doctors(data, top20_flag):
    return data[data['top20_institution'] == top20_flag]

# Function to perform aggregation and prepare ADI decile data
def prepare_aggregated_data(data):
    # Assign ADI Scores to Clinicians and aggregate data by ZIP Code and State
    adi_data = data[['ZIP Code', 'Grd_yr', 'State', 'Avg_ADI_NATRANK_2015', 'Avg_ADI_NATRANK_2020',
                     'top20_institution', 'pri_spec_grouped', 'Estimate!!SEX AND AGE!!Total population']]
    adi_data['clinician_count'] = 1

    # Aggregate data by ZIP Code, Year, State, and ADI
    adi_aggregated = adi_data.groupby(['ZIP Code', 'Grd_yr', 'State']).agg(
        total_clinicians=('clinician_count', 'sum'),
        total_population=('Estimate!!SEX AND AGE!!Total population', 'mean'),  # Use mean to avoid double-counting population
        Avg_ADI_2015=('Avg_ADI_NATRANK_2015', 'mean'),
        Avg_ADI_2020=('Avg_ADI_NATRANK_2020', 'mean')
    ).reset_index()

    # Group ADI Scores into Deciles
    adi_aggregated['ADI_decile_2015'] = pd.qcut(adi_aggregated['Avg_ADI_2015'], 10, labels=False)
    adi_aggregated['ADI_decile_2020'] = pd.qcut(adi_aggregated['Avg_ADI_2020'], 10, labels=False)

    return adi_aggregated

# Function to calculate clinician density (clinicians per 1000 population) by ADI decile
def calculate_clinician_density(data, year):
    if year == 2015:
        # Aggregate by ADI decile, taking the sum of clinicians and the mean population per decile
        adi_decile_density = data.groupby('ADI_decile_2015').agg(
            clinician_count=('total_clinicians', 'sum'),
            avg_population=('total_population', 'mean')
        ).reset_index()

        # Calculate clinicians per 1000 population based on the average population
        adi_decile_density['clinician_density'] = (adi_decile_density['clinician_count'] / adi_decile_density['avg_population']) #* 1000
        adi_decile_density.rename(columns={'ADI_decile_2015': 'ADI_decile'}, inplace=True)

    elif year == 2020:
        # Aggregate by ADI decile, taking the sum of clinicians and the mean population per decile
        adi_decile_density = data.groupby('ADI_decile_2020').agg(
            clinician_count=('total_clinicians', 'sum'),
            avg_population=('total_population', 'mean')
        ).reset_index()

        # Calculate clinicians per 1000 population based on the average population
        adi_decile_density['clinician_density'] = (adi_decile_density['clinician_count'] / adi_decile_density['avg_population']) #* 1000
        adi_decile_density.rename(columns={'ADI_decile_2020': 'ADI_decile'}, inplace=True)

    return adi_decile_density

# Function to calculate regression statistics
def calculate_regression_stats(data, x_column, y_column):
    # Fit the regression model
    X = sm.add_constant(data[x_column])
    model = sm.OLS(data[y_column], X).fit()

    # Extract regression statistics
    slope = model.params[1]
    intercept = model.params[0]
    r_squared = model.rsquared
    conf_int = model.conf_int()

    return slope, intercept, r_squared, conf_int

# Function to plot scatter plot with regression line, 95% CI, and print regression stats
def plot_scatter_with_regression_and_ci(data, x_column, y_column, title, x_label, y_label):
    plt.figure(figsize=(10, 6))

    # Scatter plot
    sns.scatterplot(x=x_column, y=y_column, data=data, s=100, color='blue')

    # Fit a regression model
    X = sm.add_constant(data[x_column])
    model = sm.OLS(data[y_column], X).fit()
    y_pred = model.predict(X)

    # Plot regression line
    plt.plot(data[x_column], y_pred, color='black', linestyle='--')

    # Plot 95% confidence interval
    pred = model.get_prediction(X)
    pred_ci = pred.conf_int()
    plt.fill_between(data[x_column], pred_ci[:, 0], pred_ci[:, 1], color='gray', alpha=0.3)

    # Add labels and title
    plt.title(title, fontsize=14)
    plt.xlabel(x_label, fontsize=12)
    plt.ylabel(y_label, fontsize=12)
    #plt.ylim(0, 1)

    plt.grid(True, linestyle='--', linewidth=0.5)
    plt.tight_layout()

    plt.show()

    # Print regression stats in specified format
    slope = model.params[1]
    intercept = model.params[0]
    r_squared = model.rsquared
    conf_int = model.conf_int()

    print(f"Slope: {slope:.4f}")
    print(f"Intercept: {intercept:.4f}")
    print(f"R-squared: {r_squared:.4f}")
    print(f"95% CI for Slope: ({conf_int.iloc[1, 0]:.4f}, {conf_int.iloc[1, 1]:.4f})")
    print(f"95% CI for Intercept: ({conf_int.iloc[0, 0]:.4f}, {conf_int.iloc[0, 1]:.4f})\n")

# Main function to run the analysis
def run_analysis_with_ci(top20_flag):
    # Step 1: Filter doctors based on top 20 flag (1 for top 20 institutions, 0 for others)
    medical_data_filtered = filter_doctors(medical_data, top20_flag)

    # Step 2: Prepare aggregated data with clinicians per 1000 population
    adi_aggregated = prepare_aggregated_data(medical_data_filtered)

    # Step 3: Calculate clinician density for 2015 and 2020
    adi_decile_density_2015 = calculate_clinician_density(adi_aggregated, 2015)
    adi_decile_density_2020 = calculate_clinician_density(adi_aggregated, 2020)

    # Step 4: Plot scatter plot with regression line and 95% CI for 2015
    plot_scatter_with_regression_and_ci(
        adi_decile_density_2015,
        'ADI_decile',
        'clinician_density',
        'Relationship Between ADI Decile and Clinician Density (2015)',
        'ADI Decile (2015)',
        'Clinicians per 1000 Population'
    )

    # Step 5: Plot scatter plot with regression line and 95% CI for 2020
    plot_scatter_with_regression_and_ci(
        adi_decile_density_2020,
        'ADI_decile',
        'clinician_density',
        'Relationship Between ADI Decile and Clinician Density (2020)',
        'ADI Decile (2020)',
        'Clinicians per 1000 Population'
    )

# Example of running the analysis with the top20_flag
run_analysis_with_ci(top20_flag=0)


